In [ ]:
import os
path = "/kaggle/input/uaskdata"
import shutil
import zipfile

# Define paths
data_path = "/kaggle/input/uaskdata"
working_path = "/kaggle/working/uaskdata"

#Ensure working directory exists
os.makedirs(working_path, exist_ok=True)

# Unzip all dataset files into /kaggle/working/
for file in os.listdir(data_path):
    if file.endswith(".zip"):
        zip_file_path = os.path.join(data_path, file)
        print(f"Extracting {zip_file_path}...")
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(working_path)

print("Dataset extracted to:", working_path)


Dataset extracted to: /kaggle/working/uaskdata


In [1]:
import os
# import shutil

# Copy dataset from read-only Kaggle input to working directory
# shutil.copytree(data_path, working_path, dirs_exist_ok=True)

print("Dataset copied to working directory!")

# Define dataset path
data_path = "/Users/shahdivyank/Desktop/cs225-final-project"

# Check the `tweet2000000` directory
tweet2000000_path = os.path.join(data_path, "tweet2000000")
if os.path.exists(tweet2000000_path):
    print("tweet2000000 exists!")
    print("Files in tweet2000000:", os.listdir(tweet2000000_path))
else:
    print("❌ tweet2000000 folder is missing!")

Dataset copied to working directory!
tweet2000000 exists!
Files in tweet2000000: ['15.txt', '14.txt', '16.txt', '17.txt', '13.txt', '12.txt', '10.txt', '11.txt', '9.txt', '8.txt', '5.txt', '4.txt', '6.txt', '7.txt', '3.txt', '2.txt', '0.txt', '1.txt', '19.txt', '18.txt']


In [2]:
ten_million = f"{data_path}/tweet10000000"
two_million = f"{data_path}/tweet2000000"
four_million = f"{data_path}/tweet4000000"
six_million = f"{data_path}/tweet6000000"
eight_million = f"{data_path}/tweet8000000"

In [3]:
import os

for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder)
    if os.path.isdir(folder_path):
        print(f" {folder}: {os.listdir(folder_path)[:5]}")  # Show first 5 files


 tweet2000000: ['15.txt', '14.txt', '16.txt', '17.txt', '13.txt']
 tweet4000000: ['29.txt', '15.txt', '14.txt', '28.txt', '16.txt']
 tweet6000000: ['29.txt', '15.txt', '14.txt', '28.txt', '16.txt']
 tweet10000000: ['29.txt', '15.txt', '14.txt', '28.txt', '16.txt']
 tweet8000000: ['29.txt', '15.txt', '14.txt', '28.txt', '16.txt']
 .git: ['config', 'objects', 'HEAD', 'info', 'logs']


In [4]:
### Object Class Definitions ###
class ObjectPass1:
    """Represents a spatial object for Pass 1 (only ID and coordinates)."""
    def __init__(self, oid, x, y):
        self.id = oid
        self.x = x
        self.y = y

class Rectangle:
    """Defines a bounding box for Quadtree nodes."""
    def __init__(self, x_min, y_min, x_max, y_max):
        self.x_min = x_min
        self.y_min = y_min
        self.x_max = x_max
        self.y_max = y_max

    def contains(self, x, y):
        """Check if (x, y) is inside the rectangle (handles precision issues)."""
        epsilon = 1e-6  # Small tolerance to prevent floating-point issues
        return (self.x_min - epsilon <= x <= self.x_max + epsilon) and (self.y_min - epsilon <= y <= self.y_max + epsilon)

class QuadTree:
    """Quadtree Node Structure."""
    def __init__(self, boundary, capacity=4):
        self.boundary = boundary  # The rectangular area covered by this node
        self.capacity = capacity  # Max objects before subdivision
        self.objects = []  # Objects stored in this node
        self.children = [None, None, None, None]  # Four quadrants
        self.is_leaf = True  # Whether this is a leaf node
        self.neighbors = []  # Store neighboring leaf nodes

In [5]:
import os
import pickle
from tqdm import tqdm

### Quadtree Construction ###
def build_quad_tree(node, objects, depth=0, max_depth=20):
    """Recursively builds a Quadtree with improved debugging and verification."""
    node.objects = objects  # Store objects in node before splitting

    # Base Case 1: Stop if max depth reached or too few objects
    if depth >= max_depth or len(objects) <= node.capacity:
        node.is_leaf = True
        return

    # Base Case 2: Stop if region size is too small
    x_min, y_min, x_max, y_max = node.boundary.x_min, node.boundary.y_min, node.boundary.x_max, node.boundary.y_max
    if abs(x_max - x_min) < 1e-6 or abs(y_max - y_min) < 1e-6:
        node.is_leaf = True
        return

    # Create Quadrants
    x_mid, y_mid = (x_min + x_max) / 2, (y_min + y_max) / 2
    quadrants = [
        Rectangle(x_min, y_mid, x_mid, y_max),  # Top-left
        Rectangle(x_mid, y_mid, x_max, y_max),  # Top-right
        Rectangle(x_min, y_min, x_mid, y_mid),  # Bottom-left
        Rectangle(x_mid, y_min, x_max, y_mid)   # Bottom-right
    ]

    node.children = [QuadTree(q, node.capacity) for q in quadrants]

    # Assign Objects to Children
    child_objects = {i: [] for i in range(4)}
    unassigned_objects = []  # Track objects that do not fit

    for obj in objects:
        assigned = False
        for i, child in enumerate(node.children):
            if child.boundary.contains(obj.x, obj.y):
                child_objects[i].append(obj)
                assigned = True
                break
        if not assigned:
            unassigned_objects.append(obj)

    # Handle Unassigned Objects
    if unassigned_objects:
        node.objects = unassigned_objects  # Keep unassigned objects in this node
        node.is_leaf = True
        return

    # Recursively Build Children
    for i, child in enumerate(node.children):
        if child_objects[i]: 
            build_quad_tree(child, child_objects[i], depth + 1, max_depth)

    node.objects = []

### Leaf Collection & Neighbor Assignment ###
def collect_leaf_nodes(node, leaf_list=None):
    """Recursively collects all leaf nodes in the Quadtree."""
    if leaf_list is None:
        leaf_list = []
    if node.is_leaf:
        leaf_list.append(node)
    else:
        for child in node.children:
            if child is not None:
                collect_leaf_nodes(child, leaf_list)
    return leaf_list

def find_neighbors_fixed(leaf_nodes):
    """Assigns neighbors to each leaf node based on shared boundaries."""
    for node in leaf_nodes:
        node.neighbors = []
        for other in leaf_nodes:
            if node == other:
                continue
            shared_x = (node.boundary.x_max == other.boundary.x_min) or (node.boundary.x_min == other.boundary.x_max)
            shared_y = (node.boundary.y_max == other.boundary.y_min) or (node.boundary.y_min == other.boundary.y_max)
            if shared_x or shared_y:
                node.neighbors.append(other)

### Save Location Table ###
def save_location_table(root, base_path, folder, batch_size=50):
    """Saves the location table for each leaf node using batch processing."""
    leaf_nodes = collect_leaf_nodes(root)
    for i in tqdm(range(0, len(leaf_nodes), batch_size), desc="Saving Location Tables", unit="batch"):
        batch = leaf_nodes[i:i+batch_size]
        for node in batch:
            if node.is_leaf:
                location_table = {obj.id: (obj.x, obj.y) for obj in node.objects}
                filename = os.path.join(base_path, f"{folder}_location_table_{id(node)}.pkl")
                with open(filename, "wb") as f:
                    pickle.dump(location_table, f)
                node.ltp = filename  

In [6]:
import os
import pickle
import glob
import numpy as np
from datetime import datetime

quadtree_times = np.zeros(5)

folders = glob.glob(os.path.join(data_path, "tweet*"))

FOLDERS = [2, 4, 6, 8, 10]

start = datetime.now()

for folder_index in range(len(folders)):
    objects_pass1 = []
    files = glob.glob(os.path.join(data_path, folders[folder_index], "*.txt"))
    
    for filename in files:
        try:
            with open(filename, "r") as file:
                for line in file:
                    parts = line.strip().split()
                    if len(parts) > 4:
                        oid, x, y = parts[0], float(parts[1]), float(parts[2])
                        objects_pass1.append(ObjectPass1(oid, x, y))
        except FileNotFoundError:
            print(f"❌ Error: File not found: {filename}")
            exit(1)

    ### Create Quadtree ###
    min_x, max_x = min(obj.x for obj in objects_pass1), max(obj.x for obj in objects_pass1)
    min_y, max_y = min(obj.y for obj in objects_pass1), max(obj.y for obj in objects_pass1)
    root_boundary = Rectangle(min_x, min_y, max_x, max_y)
    root = QuadTree(root_boundary, capacity=4)

    ### Build Quadtree ###
    build_quad_tree(root, objects_pass1, depth=0, max_depth=25)

    ### Assign Neighbors ###
    leaf_nodes = collect_leaf_nodes(root)
    find_neighbors_fixed(leaf_nodes)

    ### Save Location Tables ###
    save_location_table(root, data_path, FOLDERS[folder_index], batch_size=50)

    ### Save Quadtree ###
    with open(os.path.join(data_path, f"{FOLDERS[folder_index]}_quadtree.pkl"), "wb") as f:
        pickle.dump(root, f)

    quadtree_times[folder_index] = (datetime.now() - start).total_seconds()

### Final Debugging Check ###
leaf_nodes = collect_leaf_nodes(root)
print(f"✅ Pass 1 complete: Quadtree built.")

np.save("quadtree_times.npy", quadtree_times)


Saving Location Tables: 100%|██████████| 1/1 [00:00<00:00, 274.33batch/s]


✅ Pass 1 complete: Quadtree built.


In [6]:
class ObjectPass2:
    def __init__(self, obj_id, x, y, keywords, weights):
        self.id = obj_id
        self.x = x
        self.y = y
        self.keywords = keywords
        self.weights = weights

In [7]:
import hashlib

# Hash Function for Consistent Node Identification
def get_node_hash(node):
    key = f"{node.boundary.x_min}_{node.boundary.y_min}_{node.boundary.x_max}_{node.boundary.y_max}"
    return hashlib.md5(key.encode()).hexdigest()[:16]  # Shortened for efficiency

# Locate the Correct Leaf Node
def locate_leaf(root, x, y):
    node = root
    while not node.is_leaf:
        x_mid = (node.boundary.x_min + node.boundary.x_max) / 2
        y_mid = (node.boundary.y_min + node.boundary.y_max) / 2

        if y >= y_mid:
            if x <= x_mid:
                node = node.children[0]  # Top Left
            else:
                node = node.children[1]  # Top Right
        else:
            if x <= x_mid:
                node = node.children[2]  # Bottom Left
            else:
                node = node.children[3]  # Bottom Right
    return node

In [8]:
import os
import gzip
import json
from tqdm import tqdm

# Build and Save Textual Indexes Using JSON
def build_textual_indexes_as_json(root, base_path, folder):
    leaf_nodes = collect_leaf_nodes(root)
    json_data = {}

    print(f"✅ Processing {len(leaf_nodes)} leaf nodes for textual index...")

    for node in tqdm(leaf_nodes, desc="Building Textual Index", unit="node"):
        node_hash = get_node_hash(node)

        # If node is empty, ensure it is indexed with empty structures
        if not node.objects:
            json_data[node_hash] = {"oti": {}, "iti": {}}
            node.iti = os.path.join(base_path, "textual_index.json.gz")
            continue

        # 1) Build Object Text Index (OTI)
        text_data = {obj.id: list(zip(obj.keywords, obj.weights)) for obj in node.objects}

        # 2) Build Inverted Textual Index (ITI)
        inverted_dict = {}
        for obj in node.objects:
            for kw, w_str in zip(obj.keywords, obj.weights):
                w_val = float(w_str)
                if kw not in inverted_dict:
                    inverted_dict[kw] = []
                inverted_dict[kw].append((obj.id, w_val))

        json_data[node_hash] = {"oti": text_data, "iti": inverted_dict}
        node.iti = os.path.join(base_path, f"{folder}_textual_index.json.gz")

    #Save entire JSON at once with Gzip compression
    json_filename = os.path.join(base_path, f"{folder}_textual_index.json.gz")
    with gzip.open(json_filename, "wt", encoding="utf-8") as f:
        json.dump(json_data, f)

    print(f"✅ Saved all textual indexes in: {json_filename} (Compressed)")

In [ ]:
import os
import gzip
import json
import pickle
import numpy as np
import glob
from datetime import datetime

pass_two_times = np.zeros(5)

FOLDERS = [2, 4, 6, 8, 10]
folders = glob.glob(os.path.join(data_path, "tweet*"))

for folder_index in range(len(FOLDERS)):

    start = datetime.now()

    # Load Quadtree
    quadtree_path = os.path.join(data_path, f"{FOLDERS[folder_index]}_quadtree.pkl")
    if not os.path.exists(quadtree_path):
        raise FileNotFoundError(f"❌ Error: Quadtree file not found at {quadtree_path}. Run Pass 1 first.")

    with open(quadtree_path, "rb") as f:
        root = pickle.load(f)
    print("✅ Quadtree loaded.")

    #Step 1: Reset objects in all leaves before inserting new ones
    for leaf in collect_leaf_nodes(root):
        leaf.objects = []  # Clear previous objects

    # Step 2: Read and Process the Dataset Again (Pass 2)
    filename = os.path.join(data_path, "tweet2000000", "0.txt")
    files = glob.glob(os.path.join(data_path, folders[folder_index], "*.txt"))

    for file in files:
        objects_loaded = 0  # Debug counter

        with open(filename, "r") as file:
            for line in file:
                parts = line.strip().split()
                if len(parts) < 4:
                    print(f"Skipping invalid line: {line.strip()}")
                    continue

                oid = parts[0]
                try:
                    x = float(parts[1])
                    y = float(parts[2])
                    keyword_count = int(parts[3])
                except ValueError:
                    print(f"Error: Invalid number format in line: {line.strip()}")
                    continue

                #Read (keyword, weight) pairs
                keywords = []
                weights = []
                idx = 4
                for _ in range(keyword_count):
                    try:
                        kw = parts[idx]
                        wt = float(parts[idx + 1])  # Convert weight to float
                        keywords.append(kw)
                        weights.append(wt)
                        idx += 2
                    except (IndexError, ValueError):
                        print(f"Skipping malformed keyword-weight pair in line: {line.strip()}")
                        break

                # Create an Object for Pass 2
                obj2 = ObjectPass2(oid, x, y, keywords, weights)

                # Insert into the correct leaf node
                leaf_node = locate_leaf(root, x, y)
                leaf_node.objects.append(obj2)
                objects_loaded += 1  # Increment debug counter
    
    pass_two_times[folder_index] = (datetime.now() - start).total_seconds()

    print(f"✅ Loaded {objects_loaded} objects into Quadtree for Pass 2.")

    #Step 3: Build textual indexes using JSON storage
    build_textual_indexes_as_json(root, data_path, FOLDERS[folder_index])


✅ Quadtree loaded.
✅ Loaded 100000 objects into Quadtree for Pass 2.
✅ Processing 1 leaf nodes for textual index...


Building Textual Index: 100%|██████████| 1/1 [00:03<00:00,  3.03s/node]


✅ Saved all textual indexes in: /Users/shahdivyank/Desktop/cs225-final-project/2_textual_index.json.gz (Compressed)
✅ Quadtree loaded.
✅ Loaded 100000 objects into Quadtree for Pass 2.
✅ Processing 1 leaf nodes for textual index...


Building Textual Index: 100%|██████████| 1/1 [00:12<00:00, 12.20s/node]


✅ Saved all textual indexes in: /Users/shahdivyank/Desktop/cs225-final-project/4_textual_index.json.gz (Compressed)
✅ Quadtree loaded.
✅ Loaded 100000 objects into Quadtree for Pass 2.
✅ Processing 1 leaf nodes for textual index...


Building Textual Index: 100%|██████████| 1/1 [00:25<00:00, 25.65s/node]


✅ Saved all textual indexes in: /Users/shahdivyank/Desktop/cs225-final-project/6_textual_index.json.gz (Compressed)


In [18]:
import gzip
import json

# Load Textual Index
def load_textual_indexes(base_path):
    """Loads the JSON-based textual index from a compressed file."""
    json_filename = os.path.join(base_path, "6_textual_index.json.gz")

    if not os.path.exists(json_filename):
        raise FileNotFoundError(f"❌ Error: Textual index file not found at {json_filename}. Run Pass 2 to generate this file.")

    with gzip.open(json_filename, "rt", encoding="utf-8") as f:
        textual_index = json.load(f)

    print(f"✅ Loaded textual index from: {json_filename}")
    return textual_index

# Load the textual index
textual_index = load_textual_indexes(data_path)

# Print a few stored node hashes
stored_node_hashes = list(textual_index.keys())  # Print first 10 hashes
print("✅ Example Node Hashes stored in textual index:", stored_node_hashes)


✅ Loaded textual index from: /Users/shahdivyank/Desktop/cs225-final-project/6_textual_index.json.gz
✅ Example Node Hashes stored in textual index: ['e5c55af39ff83355']


In [ ]:
import os
import heapq
import math
import pickle
import hashlib
from collections import defaultdict

# Query Class
class Query:
    """
    Represents a spatial-textual query.
    """
    def __init__(self, loc, pos_keywords=None, neg_phrases=None, k=10, lam=0.5):
        self.loc = loc
        self.pos = pos_keywords if pos_keywords else []
        self.neg = neg_phrases if neg_phrases else []
        self.k = k
        self.lam = lam  # Lambda controls weighting between spatial & textual scores

# Hash Function for Node Identification
def get_node_hash(node):
    """Generates a unique, persistent identifier for a Quadtree node based on its boundary."""
    key = f"{node.boundary.x_min}_{node.boundary.y_min}_{node.boundary.x_max}_{node.boundary.y_max}"
    return hashlib.md5(key.encode()).hexdigest()[:16]  # Shortened for efficiency

# Distance Calculation
def euclidean_distance(a, b):
    """Computes Euclidean distance between two points."""
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def normalized_spatial_score(obj_loc, query_loc, max_dist=1000.0):
    """Computes a normalized spatial score (1 = closest, 0 = farthest)."""
    dist = euclidean_distance(obj_loc, query_loc)
    return 1.0 - min(dist / max_dist, 1.0)

# Negative Keyword Filtering
def violates_negative_phrases(obj_id, iti, neg_phrases):
    """
    Checks if the object contains any negative phrase match.
    Fix: Uses phrase-based filtering instead of individual word filtering.
    """
    if not neg_phrases:
        return False

    for phrase in neg_phrases:
        words = phrase.split()
        if all(word in iti and obj_id in [x[0] for x in iti[word]] for word in words):
            return True
    return False

# Local POWER Algorithm
def local_POWER(node, q, textual_index):
    """
    Performs local POWER query processing on a given node.
    Uses the stored textual indexes to rank objects.
    """
    node_hash = get_node_hash(node)

    if node_hash not in textual_index:
        return []

    oti = textual_index[node_hash]["oti"]
    iti = textual_index[node_hash]["iti"]

    # Step 1: Build a spatially sorted list
    spatial_list = []
    for obj_id in oti.keys():
        obj_loc = node.boundary.x_min, node.boundary.y_min  # Approximate object location
        sp_score = normalized_spatial_score(obj_loc, q.loc)
        spatial_list.append((obj_id, sp_score))

    spatial_list.sort(key=lambda x: x[1], reverse=True)

    # Step 2: Textual scoring
    def compute_textual_score(obj_id):
        score = 0.0
        for kw in q.pos:
            if kw in iti and obj_id in [x[0] for x in iti[kw]]:
                score += 1.0
        return min(score, 1.0)

    # Step 3: Compute final ranking
    local_top_k = []
    for obj_id, sp_score in spatial_list:
        txt_score = compute_textual_score(obj_id)
        if violates_negative_phrases(obj_id, iti, q.neg):
            continue
        final_score = q.lam * sp_score + (1 - q.lam) * txt_score
        heapq.heappush(local_top_k, (-final_score, obj_id))

    return heapq.nsmallest(q.k, local_top_k)

# Locate the Correct Leaf Node
def locate_leaf(root, x, y):
    """Finds the correct leaf node where an object (x, y) belongs."""
    node = root
    while not node.is_leaf:
        x_mid = (node.boundary.x_min + node.boundary.x_max) / 2
        y_mid = (node.boundary.y_min + node.boundary.y_max) / 2

        if y >= y_mid:
            if x <= x_mid:
                node = node.children[0]
            else:
                node = node.children[1]
        else:
            if x <= x_mid:
                node = node.children[2]
            else:
                node = node.children[3]
    return node

# **Batch Query Processing**
def batch_process_power(root, queries, textual_index):
    """
    Implements batch query processing for spatial-textual queries.
    Optimized for shared node access.
    """
    query_groups = defaultdict(list)

    # Step 1: Group queries by leaf node
    for query in queries:
        leaf = locate_leaf(root, query.loc[0], query.loc[1])
        query_groups[leaf].append(query)

    batch_results = {}

    # Step 2: Process queries in batches
    for leaf, grouped_queries in query_groups.items():
        node_hash = get_node_hash(leaf)

        if node_hash not in textual_index:
            continue

        results_per_query = {}
        for q in grouped_queries:
            results_per_query[q] = local_POWER(leaf, q, textual_index[node_hash])

        batch_results[node_hash] = results_per_query

    return batch_results

# Load Quadtree and Textual Index
with open(os.path.join(data_path, "6_quadtree.pkl"), "rb") as f:
    root = pickle.load(f)
print("Quadtree loaded.")

textual_index = load_textual_indexes(data_path)

# Define Sample Queries
queries = [
    Query(loc=(-21.18474656, -47.78603054), pos_keywords=["pizza"], k=5, lam=0.8),
    # Query(loc=(700.0, 300.0), pos_keywords=["burger"], neg_phrases=[], k=3, lam=0.5),
    # Query(loc=(200.0, 800.0), pos_keywords=["sushi"], neg_phrases=["bad"], k=7, lam=0.6),
]

# Run Batch Query Processing
batch_results = batch_process_power(root, queries, textual_index)

# Display Results
for node_hash, query_results in batch_results.items():
    print(f"Node Hash: {node_hash}")
    for query, results in query_results.items():
        print(f" Query at {query.loc} => {results}")


In [ ]:
#Implement the Boolean range query

#First find the leaf nodes within that range(provided xmax,xmin,ymax,ymin)

#Separately using the Positive keywords and negative keywords to find the corresponding object ids for each keyword and for each group

#From positive keyword group, we remove the overlapping obeject ids in the negative group

#output the remaining object ids as our query output
        
def boolean_range_query(root, xmin, xmax, ymin, ymax, pos_keywords, neg_keywords, textual_index, boolean = "OR"):
    """
    Implements a Boolean range query.
    
    Steps:
    1. Identify leaf nodes within the given spatial range.
    2. Retrieve object IDs separately for positive and negative keyword groups.
    3. Remove overlapping object IDs from the negative keyword group.
    4. Return the filtered object IDs.
    
    Parameters:
    - root: The root of the Quadtree.
    - xmin, xmax, ymin, ymax: The bounding box defining the query range.
    - pos_keywords: List of positive keywords.
    - neg_keywords: List of negative keywords.
    - textual_index: The inverted textual index (ITI).

    Returns:
    - A set of object IDs that match the positive keywords but exclude negative ones.
    """
    
    # Step 1: Find all leaf nodes within the range
    def get_leaf_nodes_in_range(node, xmin, xmax, ymin, ymax, result = None):
        """Recursively collect leaf nodes that intersect with the range."""
        if result is None:
            result = []
        
        if node.is_leaf:
            if not (node.boundary.x_max < xmin or node.boundary.x_min > xmax or 
                    node.boundary.y_max < ymin or node.boundary.y_min > ymax):
                result.append(node)
        else:
            for child in node.children:
                if child is not None:
                    get_leaf_nodes_in_range(child, xmin, xmax, ymin, ymax, result)
        
        return result

    leaf_nodes = get_leaf_nodes_in_range(root, xmin, xmax, ymin, ymax)

    # Step 2: Collect object IDs based on positive and negative keyword groups
    pos_object_ids = set()
    neg_object_ids = set()

    for node in leaf_nodes:
        node_hash = get_node_hash(node)
        if node_hash not in textual_index:
            continue
        
        iti = textual_index[node_hash]["iti"]

        # Collect object IDs for positive keywords        
        for kw in pos_keywords:
            if kw in iti:
                pos_object_ids.update(obj_id for obj_id, _ in iti[kw])

        # Collect object IDs for negative keywords
        for kw in neg_keywords:
            if kw in iti:
                neg_object_ids.update(obj_id for obj_id, _ in iti[kw])

    # Step 3: Remove negative keyword matches from positive object IDs
    filtered_object_ids = pos_object_ids - neg_object_ids

    return filtered_object_ids

In [ ]:
import numpy as np
from datetime import datetime

# Define spatial range (bounding box)
positive_words_times = np.zeros(5)
negative_words_times = np.zeros(5)
positive_negative_times = np.zeros(5)

xmin, xmax = -50.0, 50.0  # X-coordinate range
ymin, ymax = -50.0, 50.0  # Y-coordinate range

positive_words = ["food", "pizza", "chicago", "photos", "drink"]
negative_words = ["bad", "slow", "traffic", "dirty", "scared"]

for i in range(5):
    start = datetime.now()
    filtered_results = boolean_range_query(root, xmin, xmax, ymin, ymax, positive_words[:i], [], textual_index)
    positive_words_times[i] = (datetime.now() - start).total_seconds()

for i in range(5):
    start = datetime.now()
    filtered_results = boolean_range_query(root, xmin, xmax, ymin, ymax, [], negative_words[:i], textual_index)
    negative_words_times[i] = (datetime.now() - start).total_seconds()

for i in range(5):
    start = datetime.now()
    filtered_results = boolean_range_query(root, xmin, xmax, ymin, ymax, positive_words[:i], negative_words[:i], textual_index)
    positive_negative_times[i] = (datetime.now() - start).total_seconds()

np.save("6_positive_words_times.npy", positive_words_times)
np.save("6_negative_words_times.npy", negative_words_times)
np.save("6_positive_negative_times.npy", positive_negative_times)